In [2]:
SCRIPT_DIR = os.path.dirname('../src/')
sys.path.append(os.path.dirname(SCRIPT_DIR))

In [3]:
import warnings
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from pyod.models.knn import KNN 

from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier,
    HistGradientBoostingClassifier,
    AdaBoostClassifier
)
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

from src.wrapped import Wrapped
from src.train import cross_validate_balancead

In [4]:
warnings.filterwarnings("ignore")

wp = Wrapped(
    '../data/row/',
    '../data/processed/',
    '../data/files/'
)

# Importando dados para treinamento

In [5]:
df = wp.load_data('df_instrumentos_features_selecionadas')
df

,tonnetz0,tonnetz1,chroma1,chroma2,chroma3,chroma4,chroma5,chroma6,chroma7,mfcc0,...,spectral_band4,spectral_centroid,spectral_onset,spectral_rolloff,spectogram,mel_spectogram,zero_crossing_rate,zero_crossing,instrumento,file_name
0,0.010380,0.038218,0.513887,0.581963,0.434641,0.293949,0.317826,0.271148,0.548258,-24.170067,...,3409.640722,2600.055158,1.098243,5260.821321,-44.640682,-30.202150,0.149672,33095,accordion,000139_119040.ogg
1,0.084643,-0.035337,0.175587,0.429345,0.840043,0.384524,0.282545,0.346341,0.482365,-90.556145,...,3516.595715,2385.271420,1.360742,5225.124166,-46.575737,-59.323132,0.112181,24785,accordion,000145_172800.ogg
2,-0.030127,0.082404,0.453814,0.219808,0.228484,0.446050,0.178916,0.250981,0.327460,-106.124260,...,3256.491175,2069.850731,1.343197,4287.155824,-56.276707,-71.512909,0.096007,21230,accordion,000201_168960.ogg
3,0.157274,-0.066099,0.209729,0.338755,0.660264,0.256625,0.275209,0.180417,0.399448,-66.319801,...,3855.791675,2903.979876,1.218963,6751.133809,-42.723160,-44.649315,0.144010,31836,accordion,000212_211200.ogg
4,-0.049096,-0.029067,0.484400,0.334572,0.441629,0.679485,0.382958,0.326033,0.205775,-226.263824,...,2883.225952,1251.374287,1.339527,2263.034730,-62.227947,-78.196365,0.059804,13214,accordion,000640_49920.ogg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8471,0.070358,0.003048,0.332754,0.359639,0.473210,0.512169,0.607755,0.500391,0.378782,-87.742729,...,3664.812127,3194.922838,1.253166,6714.437434,-53.587788,-76.614616,0.134979,29855,voice,153341_184320.ogg
8472,0.006115,-0.030219,0.161793,0.190661,0.249882,0.378161,0.461520,0.332309,0.294171,-179.792145,...,2879.597305,1351.326334,1.450944,2412.643029,-52.071827,-76.029976,0.061211,13533,voice,153412_134400.ogg
8473,-0.008339,0.005653,0.589552,0.309548,0.187583,0.169132,0.240637,0.332375,0.333517,-153.614334,...,3274.875402,2685.831528,1.274003,4614.225695,-66.120743,-80.000000,0.157649,34869,voice,153478_30720.ogg
8474,0.108185,0.025366,0.615495,0.541314,0.467334,0.611199,0.791296,0.678383,0.508162,-80.250626,...,3376.494191,1428.726581,1.081204,3358.205447,-30.260586,-13.068514,0.032283,7136,voice,153986_203520.ogg


- Definindo as variaveis de treino e o target

In [44]:
le = LabelEncoder()
df['labels'] = le.fit_transform(df.instrumento)

In [43]:
X = df.drop(columns=['labels', 'instrumento','file_name'])
y = df['labels'].to_frame()

# TESTE 1: Balanceamento

- 1° Opção: Balanceamento das classes com SMOTH

In [ ]:
models = np.array([
    GaussianNB(),
    KNeighborsClassifier(), 
    DecisionTreeClassifier(), 
    RandomForestClassifier(), 
    HistGradientBoostingClassifier(),
    # CatBoostClassifier(),
    LGBMClassifier(),
    XGBClassifier(),
    MLPClassifier(),
    SVC(),
])

acuracy_models = []
for model in models:
    print(f'Model {model}')
    output = cross_validate_balancead(k=5, model=model, X=X, y=y, oversampling=True)
    print(output)
    acuracy_models.append(f'{model} {round(output * 100, 2)}')

In [7]:
acuracy_models

['GaussianNB() 25.33',
 'KNeighborsClassifier() 12.09',
 'DecisionTreeClassifier() 16.41',
 'RandomForestClassifier() 30.99',
 'HistGradientBoostingClassifier() 33.27',
 'LGBMClassifier() 33.64',
 "XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,\n              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,\n              early_stopping_rounds=None, enable_categorical=False,\n              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',\n              importance_type=None, interaction_constraints='',\n              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,\n              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,\n              missing=nan, monotone_constraints='()', n_estimators=100,\n              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',\n              predictor='auto', random_state=0, reg_alpha=0, ...) 33.53",
 'MLPClassifier() 14.98',
 'SVC() 15.94']

- Class Weight

In [8]:
models_class_weight = np.array([
    DecisionTreeClassifier(class_weight='balanced'), 
    RandomForestClassifier(class_weight='balanced', random_state = 0, n_jobs = -1), 
    LGBMClassifier(class_weight='balanced')
])

for model in models_class_weight:
    print(f'Model {model}')
    print(cross_validate_balancead(k=5, model=model, X=X, y=y, oversampling=False, weight=True))

Model DecisionTreeClassifier(class_weight='balanced')
Acuracia do split 0: 0.15742924528301888
Acuracia do split 1: 0.16224188790560473
Acuracia do split 2: 0.15457227138643068
Acuracia do split 3: 0.18171091445427728
Acuracia do split 4: 0.1728613569321534
0.165763135192297
Model RandomForestClassifier(class_weight='balanced', n_jobs=-1, random_state=0)
Acuracia do split 0: 0.3107311320754717
Acuracia do split 1: 0.31799410029498526
Acuracia do split 2: 0.328023598820059
Acuracia do split 3: 0.31976401179941005
Acuracia do split 4: 0.3380530973451327
0.32291318806701175
Model LGBMClassifier(class_weight='balanced')
Acuracia do split 0: 0.33962264150943394
Acuracia do split 1: 0.3297935103244838
Acuracia do split 2: 0.34041297935103243
Acuracia do split 3: 0.3327433628318584
Acuracia do split 4: 0.35870206489675516
0.3402549117827127


# TESTE 2: Remover Outlier

- Treinar modelo de classificação de outilier

In [9]:
detector_outilier = KNN()

new_df = df.drop(columns=['instrumento','file_name'])

detector_outilier.fit(new_df)

KNN(algorithm='auto', contamination=0.1, leaf_size=30, method='largest',
  metric='minkowski', metric_params=None, n_jobs=1, n_neighbors=5, p=2,
  radius=1.0)

- Checar numeros de outilers e não outilers<br/>
**0 (False) Não é outiler**<br/>
**1 (True) É outiler**

In [10]:
previsions = detector_outilier.labels_
np.unique(previsions, return_counts=True)

(array([0, 1]), array([7628,  848]))

- Distancia euclidiana dos registros

In [11]:
confiance_previsions = detector_outilier.decision_scores_
confiance_previsions

array([469.2569029 , 263.78344097, 234.86926869, ..., 547.08483006,
       501.47903332, 247.95356846])

- Checar os outilers

In [12]:
outilers_id = []

# pegando o indice dos registros que são outilers 
for indice in range(len(previsions)):
    if previsions[indice] == 1:
        outilers_id.append(indice)

# outilers 
outilers = df.iloc[outilers_id,:]
outilers

,tonnetz0,tonnetz1,chroma1,chroma2,chroma3,chroma4,chroma5,chroma6,chroma7,mfcc0,...,spectral_centroid,spectral_onset,spectral_rolloff,spectogram,mel_spectogram,zero_crossing_rate,zero_crossing,instrumento,file_name,labels
28,0.177880,0.099179,0.446227,0.186154,0.310970,0.289374,0.480033,0.271526,0.262448,-113.129501,...,3178.625677,1.504104,5658.636007,-64.840324,-79.991013,0.225914,49936,accordion,011547_157440.ogg,0
46,0.001389,0.058207,0.307783,0.359920,0.321503,0.340189,0.230577,0.230887,0.267941,-335.472443,...,3317.306522,1.480623,6072.413242,-37.332355,-79.453758,0.249911,55267,accordion,019554_199680.ogg,0
78,0.092169,0.133666,0.193358,0.281352,0.774958,0.293572,0.316613,0.302946,0.372028,-121.544807,...,2105.938025,1.162251,3689.097337,-49.523079,-69.013954,0.146228,32336,accordion,028377_42240.ogg,0
94,0.312066,0.246917,0.430660,0.239950,0.289489,0.156493,0.338560,0.262607,0.266463,-181.171143,...,1835.825453,0.991421,3336.597303,-63.885098,-79.566399,0.125387,27748,accordion,032445_253440.ogg,0
101,-0.083369,0.002492,0.365614,0.188457,0.210712,0.340728,0.218881,0.389993,0.138017,-118.916946,...,2495.492458,1.521361,4334.344015,-64.275818,-79.999847,0.203175,44935,accordion,035590_49920.ogg,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8408,0.011938,0.006710,0.223368,0.229950,0.303748,0.293760,0.263125,0.406126,0.506930,-5.747187,...,3037.057890,1.260896,7336.652079,-53.212646,-69.924911,0.133118,29465,voice,128693_729600.ogg,19
8435,0.074432,-0.104148,0.265711,0.153765,0.160796,0.181671,0.145868,0.225688,0.512277,-100.646721,...,2272.639669,1.200361,3904.104621,-45.581455,-76.195732,0.212792,47053,voice,142108_149760.ogg,19
8453,-0.198840,-0.026428,0.484748,0.558405,0.458368,0.481970,0.493589,0.498251,0.470169,-93.289604,...,1810.905441,1.761557,4142.593589,-45.505581,-29.093159,0.037906,8383,voice,147088_72960.ogg,19
8455,-0.040879,0.077836,0.445147,0.216153,0.162184,0.219060,0.224821,0.265225,0.208129,-224.087448,...,2759.989165,1.015649,6755.730224,-27.749546,-59.993065,0.164289,36301,voice,147642_53760.ogg,19


In [13]:
outilers.instrumento.value_counts()

flute                128
mallet_percussion    127
cymbals               85
synthesizer           84
violin                59
drums                 52
organ                 47
bass                  40
accordion             36
guitar                34
voice                 29
ukulele               21
banjo                 19
piano                 18
mandolin              16
clarinet              15
trombone              12
saxophone             11
trumpet                8
cello                  7
Name: instrumento, dtype: int64

- Agora vamos remover os outilers da nossa base

In [14]:
filter_id = []

# pegando indice dos registros que não são outilers
for indice in range(len(previsions)):
    if previsions[indice] == 0:
        filter_id.append(indice)

# removendo outilers  
df_train = df.iloc[filter_id,:]
df_train

,tonnetz0,tonnetz1,chroma1,chroma2,chroma3,chroma4,chroma5,chroma6,chroma7,mfcc0,...,spectral_centroid,spectral_onset,spectral_rolloff,spectogram,mel_spectogram,zero_crossing_rate,zero_crossing,instrumento,file_name,labels
0,0.010380,0.038218,0.513887,0.581963,0.434641,0.293949,0.317826,0.271148,0.548258,-24.170067,...,2600.055158,1.098243,5260.821321,-44.640682,-30.202150,0.149672,33095,accordion,000139_119040.ogg,0
1,0.084643,-0.035337,0.175587,0.429345,0.840043,0.384524,0.282545,0.346341,0.482365,-90.556145,...,2385.271420,1.360742,5225.124166,-46.575737,-59.323132,0.112181,24785,accordion,000145_172800.ogg,0
2,-0.030127,0.082404,0.453814,0.219808,0.228484,0.446050,0.178916,0.250981,0.327460,-106.124260,...,2069.850731,1.343197,4287.155824,-56.276707,-71.512909,0.096007,21230,accordion,000201_168960.ogg,0
3,0.157274,-0.066099,0.209729,0.338755,0.660264,0.256625,0.275209,0.180417,0.399448,-66.319801,...,2903.979876,1.218963,6751.133809,-42.723160,-44.649315,0.144010,31836,accordion,000212_211200.ogg,0
4,-0.049096,-0.029067,0.484400,0.334572,0.441629,0.679485,0.382958,0.326033,0.205775,-226.263824,...,1251.374287,1.339527,2263.034730,-62.227947,-78.196365,0.059804,13214,accordion,000640_49920.ogg,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8470,0.049317,0.016959,0.431032,0.433562,0.439329,0.472623,0.436766,0.458280,0.480054,2.422542,...,2542.887346,1.234712,5289.648834,-53.747391,-70.919098,0.141731,31319,voice,151680_126720.ogg,19
8472,0.006115,-0.030219,0.161793,0.190661,0.249882,0.378161,0.461520,0.332309,0.294171,-179.792145,...,1351.326334,1.450944,2412.643029,-52.071827,-76.029976,0.061211,13533,voice,153412_134400.ogg,19
8473,-0.008339,0.005653,0.589552,0.309548,0.187583,0.169132,0.240637,0.332375,0.333517,-153.614334,...,2685.831528,1.274003,4614.225695,-66.120743,-80.000000,0.157649,34869,voice,153478_30720.ogg,19
8474,0.108185,0.025366,0.615495,0.541314,0.467334,0.611199,0.791296,0.678383,0.508162,-80.250626,...,1428.726581,1.081204,3358.205447,-30.260586,-13.068514,0.032283,7136,voice,153986_203520.ogg,19


- Treinar modelos sem outlier e balanceameto

In [15]:
X_remove_outilers = df_train.drop(columns=['labels', 'instrumento','file_name'])
y_remove_outilers = df_train['labels'].to_frame()

models = np.array([
    GaussianNB(),
    KNeighborsClassifier(), 
    DecisionTreeClassifier(), 
    RandomForestClassifier(), 
    HistGradientBoostingClassifier(),
    LGBMClassifier(),
    XGBClassifier(),
    MLPClassifier(),
    SVC(),
])

acuracy_models_sem_outilers = []
for model in models:
    print(f'Model {model}')
    output = cross_validate_balancead(k=5, 
                                    model=model,
                                    X=X_remove_outilers, 
                                    y=y_remove_outilers, oversampling=True)
    print(output)
    acuracy_models_sem_outilers.append(f'{model} {round(output * 100, 2)}')

Model GaussianNB()
Acuracia do split 0: 0.24639580602883354
Acuracia do split 1: 0.24967234600262123
Acuracia do split 2: 0.2490170380078637
Acuracia do split 3: 0.2583606557377049
Acuracia do split 4: 0.259672131147541
0.25262359538491286
Model KNeighborsClassifier()
Acuracia do split 0: 0.1107470511140236
Acuracia do split 1: 0.11336828309305373
Acuracia do split 2: 0.12778505897771952
Acuracia do split 3: 0.1062295081967213
Acuracia do split 4: 0.12393442622950819
0.11641286552220527
Model DecisionTreeClassifier()
Acuracia do split 0: 0.17234600262123198
Acuracia do split 1: 0.16841415465268678
Acuracia do split 2: 0.17496723460026212
Acuracia do split 3: 0.16983606557377048
Acuracia do split 4: 0.1763934426229508
0.17239138001418045
Model RandomForestClassifier()
Acuracia do split 0: 0.2948885976408912
Acuracia do split 1: 0.3073394495412844
Acuracia do split 2: 0.30799475753604194
Acuracia do split 3: 0.31081967213114753
Acuracia do split 4: 0.32131147540983607
0.3084707904518402


In [16]:
acuracy_models_sem_outilers

['GaussianNB() 25.26',
 'KNeighborsClassifier() 11.64',
 'DecisionTreeClassifier() 17.24',
 'RandomForestClassifier() 30.85',
 'HistGradientBoostingClassifier() 33.53',
 'LGBMClassifier() 33.48',
 "XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,\n              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,\n              early_stopping_rounds=None, enable_categorical=False,\n              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',\n              importance_type=None, interaction_constraints='',\n              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,\n              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,\n              missing=nan, monotone_constraints='()', n_estimators=100,\n              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',\n              predictor='auto', random_state=0, reg_alpha=0, ...) 32.94",
 'MLPClassifier() 13.61',
 'SVC() 17.13']

- Treinar modelos sem outlier  e com pesos

In [17]:
models_class_weight = np.array([
    DecisionTreeClassifier(class_weight='balanced'), 
    RandomForestClassifier(class_weight='balanced', random_state = 0, n_jobs = -1), 
    LGBMClassifier(class_weight='balanced')
])

acuracy_models_sem_outilers = []
for model in models_class_weight:
    print(f'Model {model}')
    output = cross_validate_balancead(k=5, model=model, X=X, y=y, oversampling=True)
    print(output)
    acuracy_models_sem_outilers.append(f'{model} {round(output * 100, 2)}')

Model DecisionTreeClassifier(class_weight='balanced')
Acuracia do split 0: 0.15153301886792453
Acuracia do split 1: 0.17581120943952802
Acuracia do split 2: 0.16460176991150444
Acuracia do split 3: 0.16106194690265488
Acuracia do split 4: 0.17463126843657817
0.165527842711638
Model RandomForestClassifier(class_weight='balanced', n_jobs=-1, random_state=0)
Acuracia do split 0: 0.30483490566037735
Acuracia do split 1: 0.31032448377581123
Acuracia do split 2: 0.32271386430678467
Acuracia do split 3: 0.3014749262536873
Acuracia do split 4: 0.3286135693215339
0.31359234986363893
Model LGBMClassifier(class_weight='balanced')
Acuracia do split 0: 0.33254716981132076
Acuracia do split 1: 0.3309734513274336
Acuracia do split 2: 0.3339233038348083
Acuracia do split 3: 0.3315634218289086
Acuracia do split 4: 0.3528023598820059
0.3363619413368954


In [18]:
acuracy_models_sem_outilers

["DecisionTreeClassifier(class_weight='balanced') 16.55",
 "RandomForestClassifier(class_weight='balanced', n_jobs=-1, random_state=0) 31.36",
 "LGBMClassifier(class_weight='balanced') 33.64"]

TESTE 3: Features de Maior Correlação

In [7]:
df.corr()

,tonnetz0,tonnetz1,chroma1,chroma2,chroma3,chroma4,chroma5,chroma6,chroma7,mfcc0,...,spectral_band2,spectral_band3,spectral_band4,spectral_centroid,spectral_onset,spectral_rolloff,spectogram,mel_spectogram,zero_crossing_rate,zero_crossing
tonnetz0,1.000000,0.062140,-0.097181,-0.170037,0.169987,-0.225499,0.160613,-0.176057,-0.113787,-0.029714,...,-0.060952,-0.060755,-0.058103,-0.061619,-0.092151,-0.058551,-0.062679,-0.076992,-0.051588,-0.051571
tonnetz1,0.062140,1.000000,0.242571,-0.165899,0.098279,-0.093905,-0.187621,0.059018,-0.242730,-0.062356,...,-0.076892,-0.074494,-0.069979,-0.076872,-0.062285,-0.073258,-0.010867,-0.012354,-0.065814,-0.065818
chroma1,-0.097181,0.242571,1.000000,0.494096,0.107322,0.117760,0.108515,0.168467,0.075469,0.146132,...,0.133029,0.119271,0.104452,0.108212,0.097569,0.130602,0.182205,0.227982,0.069222,0.069246
chroma2,-0.170037,-0.165899,0.494096,1.000000,0.499016,0.331268,0.141218,0.179724,0.254288,0.186089,...,0.181963,0.166170,0.148799,0.157707,0.126737,0.179851,0.198460,0.256889,0.099693,0.099708
chroma3,0.169987,0.098279,0.107322,0.499016,1.000000,0.532327,0.132038,0.105914,0.144183,0.172337,...,0.160707,0.145982,0.130206,0.133901,0.083847,0.157737,0.145190,0.198820,0.074730,0.074740
chroma4,-0.225499,-0.093905,0.117760,0.331268,0.532327,1.000000,0.499493,0.303442,0.216477,0.181720,...,0.180050,0.165171,0.147500,0.143578,0.117159,0.170458,0.200901,0.268462,0.071936,0.071957
chroma5,0.160613,-0.187621,0.108515,0.141218,0.132038,0.499493,1.000000,0.550665,0.225242,0.209649,...,0.158948,0.143825,0.127655,0.121618,0.100378,0.150782,0.201781,0.257689,0.060857,0.060870
chroma6,-0.176057,0.059018,0.168467,0.179724,0.105914,0.303442,0.550665,1.000000,0.519934,0.185260,...,0.167243,0.156626,0.143323,0.117864,0.121308,0.150682,0.239641,0.304158,0.044637,0.044646
chroma7,-0.113787,-0.242730,0.075469,0.254288,0.144183,0.216477,0.225242,0.519934,1.000000,0.198951,...,0.205535,0.188767,0.169222,0.163844,0.133441,0.196320,0.209530,0.283768,0.094836,0.094854
mfcc0,-0.029714,-0.062356,0.146132,0.186089,0.172337,0.181720,0.209649,0.185260,0.198951,1.000000,...,0.547994,0.504851,0.459083,0.498269,0.236631,0.546795,0.288350,0.265571,0.318793,0.318687


In [14]:
df_corr = df.corr().unstack().reset_index().dropna()
df_corr.rename(columns = {'level_0': 'features_a', 'level_1': 'features_b', 0:'correlacao'}, inplace = True)
df_corr

,features_a,features_b,correlacao
0,tonnetz0,tonnetz0,1.000000
1,tonnetz0,tonnetz1,0.062140
2,tonnetz0,chroma1,-0.097181
3,tonnetz0,chroma2,-0.170037
4,tonnetz0,chroma3,0.169987
...,...,...,...
1931,zero_crossing,spectral_rolloff,0.732824
1932,zero_crossing,spectogram,-0.047536
1933,zero_crossing,mel_spectogram,-0.066029
1934,zero_crossing,zero_crossing_rate,0.999999


- Correlação Positia

In [52]:
corr_posit = df_corr.query('correlacao > 0.7 and correlacao < 1.0')
corr_posit

,features_a,features_b,correlacao
1531,spectral_band2,spectral_band3,0.977866
1532,spectral_band2,spectral_band4,0.925967
1533,spectral_band2,spectral_centroid,0.864482
1535,spectral_band2,spectral_rolloff,0.927909
1574,spectral_band3,spectral_band2,0.977866
1576,spectral_band3,spectral_band4,0.984122
1577,spectral_band3,spectral_centroid,0.771820
1579,spectral_band3,spectral_rolloff,0.837256
1618,spectral_band4,spectral_band2,0.925967
1619,spectral_band4,spectral_band3,0.984122


In [78]:
X =  df[corr_posit.features_a.unique()]
y = df['labels'].to_frame()

models = np.array([
    GaussianNB(),
    KNeighborsClassifier(), 
    DecisionTreeClassifier(), 
    RandomForestClassifier(), 
    HistGradientBoostingClassifier(),
    LGBMClassifier(),
    XGBClassifier(),
    MLPClassifier(),
    SVC(),
])

acuracy_models = []
for model in models:
    print(f'Model {model}')
    output = cross_validate_balancead(k=5, model=model, X=X, y=y)
    print(output)
    acuracy_models.append(f'{model} {round(output * 100, 2)}')

Model GaussianNB()
Acuracia do split 0: 0.17275943396226415
Acuracia do split 1: 0.17345132743362832
Acuracia do split 2: 0.18525073746312684
Acuracia do split 3: 0.16519174041297935
Acuracia do split 4: 0.17994100294985252
0.17531884844437023
Model KNeighborsClassifier()
Acuracia do split 0: 0.13266509433962265
Acuracia do split 1: 0.14454277286135694
Acuracia do split 2: 0.1480825958702065
Acuracia do split 3: 0.12861356932153392
Acuracia do split 4: 0.1504424778761062
0.14086930205376524
Model DecisionTreeClassifier()
Acuracia do split 0: 0.12971698113207547
Acuracia do split 1: 0.1256637168141593
Acuracia do split 2: 0.12743362831858407
Acuracia do split 3: 0.11268436578171091
Acuracia do split 4: 0.1410029498525074
0.12730032837980743
Model RandomForestClassifier()
Acuracia do split 0: 0.18160377358490565
Acuracia do split 1: 0.19233038348082596
Acuracia do split 2: 0.20648967551622419
Acuracia do split 3: 0.19233038348082596
Acuracia do split 4: 0.19646017699115045
0.193842878610

In [79]:
acuracy_models

['GaussianNB() 17.53',
 'KNeighborsClassifier() 14.09',
 'DecisionTreeClassifier() 12.73',
 'RandomForestClassifier() 19.38',
 'HistGradientBoostingClassifier() 19.41',
 'LGBMClassifier() 19.74',
 "XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,\n              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,\n              early_stopping_rounds=None, enable_categorical=False,\n              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',\n              importance_type=None, interaction_constraints='',\n              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,\n              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,\n              missing=nan, monotone_constraints='()', n_estimators=100,\n              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',\n              predictor='auto', random_state=0, reg_alpha=0, ...) 20.16",
 'MLPClassifier() 10.78',
 'SVC() 18.13']

- Correlação Negativa

In [72]:
corr_negat = df_corr.query('correlacao < 0 and correlacao < -0.66')
corr_negat

,features_a,features_b,correlacao
474,mfcc1,spectral_band2,-0.843059
475,mfcc1,spectral_band3,-0.796348
476,mfcc1,spectral_band4,-0.725181
477,mfcc1,spectral_centroid,-0.872413
479,mfcc1,spectral_rolloff,-0.856605
482,mfcc1,zero_crossing_rate,-0.663971
483,mfcc1,zero_crossing,-0.664032
1506,spectral_band2,mfcc1,-0.843059
1550,spectral_band3,mfcc1,-0.796348
1594,spectral_band4,mfcc1,-0.725181


In [75]:
len(corr_negat.features_b.unique())

8

In [76]:
X =  df[corr_negat.features_b.unique()]
y = df['labels'].to_frame()

models = np.array([
    GaussianNB(),
    KNeighborsClassifier(), 
    DecisionTreeClassifier(), 
    RandomForestClassifier(), 
    HistGradientBoostingClassifier(),
    LGBMClassifier(),
    XGBClassifier(),
    MLPClassifier(),
    SVC(),
])

acuracy_models = []
for model in models:
    print(f'Model {model}')
    output = cross_validate_balancead(k=5, model=model, X=X, y=y)
    print(output)
    acuracy_models.append(f'{model} {round(output * 100, 2)}')

Model GaussianNB()
Acuracia do split 0: 0.15625
Acuracia do split 1: 0.15988200589970503
Acuracia do split 2: 0.17581120943952802
Acuracia do split 3: 0.15693215339233038
Acuracia do split 4: 0.15693215339233038
0.16116150442477875
Model KNeighborsClassifier()
Acuracia do split 0: 0.13266509433962265
Acuracia do split 1: 0.14454277286135694
Acuracia do split 2: 0.14690265486725665
Acuracia do split 3: 0.13038348082595871
Acuracia do split 4: 0.15221238938053097
0.14134127845494518
Model DecisionTreeClassifier()
Acuracia do split 0: 0.14150943396226415
Acuracia do split 1: 0.1392330383480826
Acuracia do split 2: 0.13805309734513274
Acuracia do split 3: 0.13097345132743363
Acuracia do split 4: 0.14277286135693215
0.13850837646796904
Model RandomForestClassifier()
Acuracia do split 0: 0.1857311320754717
Acuracia do split 1: 0.20235988200589972
Acuracia do split 2: 0.22654867256637168
Acuracia do split 3: 0.17817109144542773
Acuracia do split 4: 0.20766961651917404
0.20009607892246897
Mode

In [77]:
acuracy_models

['GaussianNB() 16.12',
 'KNeighborsClassifier() 14.13',
 'DecisionTreeClassifier() 13.85',
 'RandomForestClassifier() 20.01',
 'HistGradientBoostingClassifier() 19.94',
 'LGBMClassifier() 20.14',
 "XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,\n              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,\n              early_stopping_rounds=None, enable_categorical=False,\n              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',\n              importance_type=None, interaction_constraints='',\n              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,\n              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,\n              missing=nan, monotone_constraints='()', n_estimators=100,\n              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',\n              predictor='auto', random_state=0, reg_alpha=0, ...) 19.75",
 'MLPClassifier() 10.64',
 'SVC() 18.19']

- Se combinarmos as duas abordagens ?

In [117]:
features = np.array(['spectral_band2', 'spectral_band3', 'spectral_band4',
                     'spectral_centroid', 'spectral_rolloff', 'zero_crossing_rate',
                     'zero_crossing', 'mfcc1', 'mfcc1', 'mfcc1', 'mfcc1', 'mfcc1',
                     'spectral_band3', 'spectral_band4', 'spectral_centroid',
                     'spectral_rolloff', 'spectral_band2', 'spectral_band4',
                     'spectral_centroid', 'spectral_rolloff', 'spectral_band2',
                     'spectral_band3', 'spectral_rolloff', 'spectral_band2',
                     'spectral_band3', 'spectral_rolloff', 'zero_crossing_rate',
                     'zero_crossing', 'spectral_band2', 'spectral_band3',
                     'spectral_band4', 'spectral_centroid', 'zero_crossing_rate',
                     'zero_crossing', 'mel_spectogram', 'spectogram',
                     'spectral_centroid', 'spectral_rolloff', 'zero_crossing',
                     'spectral_centroid', 'spectral_rolloff', 'zero_crossing_rate'])

features = np.unique(features)

In [118]:
np.unique(features)

array(['mel_spectogram', 'mfcc1', 'spectogram', 'spectral_band2',
       'spectral_band3', 'spectral_band4', 'spectral_centroid',
       'spectral_rolloff', 'zero_crossing', 'zero_crossing_rate'],
      dtype='<U18')

In [119]:
X = df[features]
y = df['labels'].to_frame()

models = np.array([
    GaussianNB(),
    KNeighborsClassifier(), 
    DecisionTreeClassifier(), 
    RandomForestClassifier(), 
    HistGradientBoostingClassifier(),
    LGBMClassifier(),
    XGBClassifier(),
    MLPClassifier(),
    SVC(),
])

acuracy_models = []
for model in models:
    print(f'Model {model}')
    output = cross_validate_balancead(k=5, model=model, X=X, y=y)
    print(output)
    acuracy_models.append(f'{model} {round(output * 100, 2)}')

Model GaussianNB()
Acuracia do split 0: 0.1733490566037736
Acuracia do split 1: 0.16932153392330385
Acuracia do split 2: 0.18761061946902655
Acuracia do split 3: 0.17168141592920355
Acuracia do split 4: 0.1775811209439528
0.1759087493738521
Model KNeighborsClassifier()
Acuracia do split 0: 0.13089622641509435
Acuracia do split 1: 0.14454277286135694
Acuracia do split 2: 0.1480825958702065
Acuracia do split 3: 0.12920353982300886
Acuracia do split 4: 0.15103244837758112
0.14075151666944957
Model DecisionTreeClassifier()
Acuracia do split 0: 0.12794811320754718
Acuracia do split 1: 0.14985250737463127
Acuracia do split 2: 0.143952802359882
Acuracia do split 3: 0.13510324483775812
Acuracia do split 4: 0.1486725663716814
0.1411058468303
Model RandomForestClassifier()
Acuracia do split 0: 0.21580188679245282
Acuracia do split 1: 0.2153392330383481
Acuracia do split 2: 0.23008849557522124
Acuracia do split 3: 0.20294985250737463
Acuracia do split 4: 0.2182890855457227
0.21649371069182394
Mod